In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keybert import KeyBERT
import time
from tqdm.notebook import tqdm

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
sds_text = pd.read_csv('../data/sds_text.csv')

<IPython.core.display.Javascript object>

In [4]:
sds_text = sds_text.set_index('Unnamed: 0')

In [5]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop
             and not token.is_digit
             and not token.is_punct
             and not token.is_space]
    return tokens

In [6]:

sds_text['tokens'] = sds_text['episode_split_text'].apply(preprocess_text)

In [7]:
sds_text

,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_day,host_episode,speaker,episode_split_text,tokens
Unnamed: 0,,,,,,,,,,,,,
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,This is episode number one with ex-chemical e...,"[episode, number, ex, chemical, engineer, data..."
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,"Hey guys, welcome to the Podcast. I’ve got Ru...","[hey, guy, welcome, podcast, get, ruben, kogel..."
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,Thank you! Thanks for having me over. I’m doi...,"[thank, thank, have, great]"
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,Awesome. It’s great to hear you and for those...,"[awesome, great, hear, know, meet, ruben, time..."
0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,"Yeah sure. So, I’m the senior manager of anal...","[yeah, sure, senior, manager, analytic, strate..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Yeah, right. Yeah, as I mean, it actually, it ...","[yeah, right, yeah, mean, actually, show, know..."
681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Matar Haller,Happy to.,[happy]
681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Nice. Well, yeah, so you mentioned potentially...","[nice, yeah, mention, potentially, future, sou..."


In [ ]:
sds_text.info()

In [ ]:
sds_processed = pd.DataFrame(sds_text.groupby('episode_number')['episode_split_text'].agg(lambda x: ' '.join(x)))

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_split_text'].loc[683]

In [ ]:
def keyword_extractor(token):
    doc = token
    kw_model = KeyBERT()
    keyword = ''
    stopwords = list(STOP_WORDS)
    for keyword in doc:
        keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1, 1), stop_words=stopwords)
    return keywords

In [ ]:
sds_processed['keywords'] = sds_processed['episode_split_text'].apply(keyword_extractor)

In [ ]:
keywords

In [ ]:
stopwords = list(STOP_WORDS)

In [ ]:
# https://maartengr.github.io/BERTopic/index.html